In [ ]:
#Putting all the imports in one place for readability
import numpy as np
import torch
from torch import nn
import random
from collections import Counter
import Anotation
import copy
from allennlp.modules.conditional_random_field import ConditionalRandomField as CRF
from torcheval.metrics.functional import multiclass_confusion_matrix as MCM

# Setting seeds to ensure reproducibility of results

random.seed(666)
np.random.seed(666)
torch.manual_seed(666)

In [ ]:
#Extracts the data into 2 lists of lists, one with the tokens another with the tags


def extractData(filePath):
    """
    Returns:tuple: A tuple containing input data (list of lists of words), tags (list of lists of tags),
    and metadata (list of tuples containing newdoc_id, sent_id, and text).
    """
    wordsData = []
    tagsData = []
    currentSent = None
    with open(filePath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith("# sent_id"):
                sentId = line.split("= ")[1]
            elif line.startswith("#"):
                continue
            elif line:                
                parts = line.split('\t')
                word = parts[1]
                tag = parts[2]
                if sentId != currentSent:
                    currentSent = sentId
                    wordsData.append([word])
                    tagsData.append([tag])
                else:
                    wordsData[-1].append(word)
                    tagsData[-1].append(tag)
    return wordsData, tagsData

# Example usage:
#file_path = "../Data/UniversalNER/train/en_ewt-ud-train.iob2"
#words_data, tags_data = extractData(file_path)
# for words, tags in zip(words_data, tags_data):
#     print("Words:", words)
#     print("Tags:", tags)
#     print()

In [ ]:
#Converts the Data into a tensor for use by the model

def convertDataShape(data, vocabulary = {}, labels = [], training = True, PADDING_TOKEN = '<PAD>', START_TOKEN = '<START>', STOP_TOKEN = '<END>', UNKNOWN_TOKEN = '<UNK>'):
    """
    If training is enabled creates a vocabulary of all words in a list. Otherwise, a vocabulary should be passed.
    Does the same with the labels.
    Creates a matrix of sentences and positions, where each value indicates a word via its index in the vocabulary.
    Creates another matrix of sentences and positions, where the values indicate a label.
    '<PAD>' or another user defined token is used as padding for short sentences. Will also act as an unknown token, if not training, it is assumed to be in vocabulary.
    Returns, the vocabulary, the labels and the two matrices.
    
    Input:
    data          - (string list * string list) list - List of sentences. Each sentence is a tuple of two lists. The first is a list of words, the second a list of labels.
    vocabulary    - string : int dictionary          - Dictionary of words in the vocabulary, values are the indices. Should be provided if not training. Defaults to empty dict.
    labels        - string : int dictionary          - Dictionary of labels to classify, values are the indices. Should be provided if not training. Defaults to empty dict.
    training      - boolean                          - Boolean variable deffining whether training is taking place, if yes then a new vocabulary will be created. Defaults to yes.
    PADDING_TOKEN - string                           - Token to be used as padding. Default is provided
    START_TOKEN   - string                           - Token to be used as marker for the start of the sentence. Default is provided
    STOP_TOKEN    - string                           - Token to be used as marker for the end of the sentence. Default is provided
    UNKNOWN_TOKEN - string                           - Token to be used as the unknown token. Default is provided
    
    Output:
    Xmatrix       - 2D torch.tensor                  - 2d torch tensor containing the index of the word in the sentence in the vocabulary
    Ymatrix       - 2D torch.tensor                  - 2d torch tensor containing the index of the label in the sentence in the labels
    vocabulary    - string : int dictionary          - Dictionary of words, with indices as values, used for training.
    labels        - string : int dictionary          - Dictionary of all the labels, with indices as values, used for classification. (all the labels are expected to be present in the training data, or in other words, the label list provided should be exhaustive)
    """


    if training:
        vocabList = sorted(set(word for sentence, _ in data for word in sentence))
        
        #In order to be able to work with unknown words in the future, we turn some of the least common words into unknown words so we can train on them
        #This is done by removing them from the vocab list before creating the dictionary
        vocabCount = Counter([word for sentence, _ in data for word in sentence])
        UNKNOWN_RATIO = 5 #This should be percentage of tokens we want to turn into Unknown tokens, the least common tokens will be used
        cutoff = int(len(vocabList) / (100 / UNKNOWN_RATIO)) + 1
        removeList = vocabCount.most_common()[:-cutoff:-1]
        for i in removeList:
            vocabList.remove(i[0])

        # Adding the special tokens in the first positions after the least common have been removed and creating the dictionaries
        vocabList = [PADDING_TOKEN, START_TOKEN, STOP_TOKEN, UNKNOWN_TOKEN] + vocabList
        vocabulary = {word: i for i, word in enumerate(vocabList)}
        labelList = [PADDING_TOKEN, START_TOKEN, STOP_TOKEN] + sorted(set(label for _, sentenceLabels in data for label in sentenceLabels))
        labels = {label: i for i, label in enumerate(labelList)}
    
    # Adding two to the max len in order to accomodate the introduction of start and end tokens
    maxLen = max(len(sentence) for sentence, _ in data) + 2
    Xmatrix = np.zeros((len(data), maxLen), dtype=int)
    Ymatrix = np.zeros((len(data), maxLen), dtype=int)

    for i, (sentence, sentenceLabels) in enumerate(data):
        #Set the first token as the start token (assumes it's index is 1)
        Xmatrix[i, 0] = 1
        Ymatrix[i, 0] = 1
        #Set all the indices to the correct index, with the unknown token as default
        for j, word in enumerate(sentence):
            Xmatrix[i, j+1] = vocabulary.get(word, vocabulary[UNKNOWN_TOKEN])
        for j, label in enumerate(sentenceLabels):
            Ymatrix[i, j+1] = labels.get(label, labels[START_TOKEN])
            lastWord = j         
        # Sets the token after the last word as en end token
        Xmatrix[i, lastWord + 2] = 2
        Ymatrix[i, lastWord + 2] = 2
    
    return torch.tensor(Xmatrix, dtype=torch.long), torch.tensor(Ymatrix, dtype=torch.long), vocabulary, labels

# two first sentences of EWT training dataset so that quickdebugging can be run



trainingDebugSen = [["Where", "in", "the", "world", "is", "Iguazu", "?"], ["Iguazu", "Falls"]]
trainingDebugTags = [["O", "O", "O", "O", "O", "B-LOC", "O"], ["B-LOC", "I-LOC"]]

dataDebug, labelsDebug, vocabDebug, tagsDebug = convertDataShape(list(zip(trainingDebugSen, trainingDebugTags)))
print(dataDebug)
print(labelsDebug)
print(vocabDebug)
print(tagsDebug)

In [ ]:
class baselineModel(torch.nn.Module):
    def __init__(self, nWords, tags, dimEmbed, dimHidden, constraints, dropOut):
        super().__init__()
        self.dimEmbed = dimEmbed
        self.dimHidden = dimHidden
        self.vocabSize = nWords
        self.tagSetSize = len(tags)

        self.embed = nn.Embedding(nWords, dimEmbed)
        self.LSTM = nn.LSTM(dimEmbed, dimHidden, bidirectional=True, dropout = dropOut)
        self.linear = nn.Linear(dimHidden * 2, self.tagSetSize)
        

        # Initialize the CRF layer
        self.CRF = CRF(self.tagSetSize, constraints = constraints, include_start_end_transitions=True)

    def forwardTrain(self, inputData, labels):
        # Embedding and LSTM layers
        wordVectors = self.embed(inputData)
        lstmOut, _ = self.LSTM(wordVectors)
        
        # Linear layer
        emissions = self.linear(lstmOut)
        
        # CRF layer to compute the log likelihood loss
        log_likelihood = self.CRF(emissions, labels)
        
        # The loss is the negative log-likelihood
        loss = -log_likelihood
        return loss
        
    def forwardPred(self, inputData):
        # Embedding and LSTM layers
        wordVectors = self.embed(inputData)
        lstmOut, _ = self.LSTM(wordVectors)
        
        # Linear layer
        emissions = self.linear(lstmOut)
        
        # Decode the best path
        best_paths = self.CRF.viterbi_tags(emissions)
        
        # Extract the predicted tags from the paths
        predictions = [path for path, score in best_paths]
        return predictions


In [ ]:

def saveToIob2(words, labels, outputFilePath):
    """
    Save words and their corresponding labels in IOB2 format.

    Args:
    words (list): List of lists containing words.
    labels (list): List of lists containing labels.
    output_file (str): Path to the output IOB2 file.
    """
    with open(outputFilePath, 'w', encoding='utf-8') as file:
        for i in range(len(words)):
            for j in range(len(words[i])):
                line = f"{j+1}\t{words[i][j]}\t{labels[i][j]}\n"
                file.write(line)
            file.write('\n')

In [ ]:
#Small dataset
# two first sentences of EWT training dataset so that quickdebugging can be run

tags = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]

trainingDebugSen = [["Where", "in", "the", "world", "is", "Iguazu", "?"], ["Iguazu", "Falls"]]
trainingDebugTags = [["O", "O", "O", "O", "O", "B-LOC", "O"], ["B-LOC", "I-LOC"]]

dataDebug, labelsDebug, vocabDebug, tagsDebug = convertDataShape(list(zip(trainingDebugSen, trainingDebugTags)))

In [ ]:
#Test train
#Quick traininig script on the debug dataset

DIM_EMBEDDING = 100
LSTM_HIDDEN = 50
LEARNING_RATE = 0.01
EPOCHS = 5

random.seed(666)
np.random.seed(666)
torch.manual_seed(666)

constraint_type = None

model = baselineModel(len(vocabDebug), tagsDebug, DIM_EMBEDDING, LSTM_HIDDEN, constraint_type, 0)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()
    
    optimizer.zero_grad()
    loss = model.forwardTrain(dataDebug, labelsDebug)
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch}, Loss: {loss.item()}")


In [ ]:
#Getting predicitons and checking accuracy


with torch.no_grad():
    predictsDebug = model.forwardPred(dataDebug)

confMat = MCM(torch.flatten(torch.tensor(predictsDebug, dtype=torch.long)), torch.flatten(labelsDebug), num_classes = len(tagsDebug))

acc = torch.trace(confMat[1:,1:])/torch.sum(confMat[1:,1:]) #Taking away the first collumn and first row, because those correspond to the padding token and we don't care
acc

In [ ]:
# Loading all the training data sets
#pass train data size
def loadingAllData(train = 0.8):
    filePathTrain = "../Data/UniversalNER/train/"   
    wordsData = []
    tagsData = []
    datasets = ["da_ddt", "en_ewt", "hr_set", "pt_bosque", "sk_snk", "sr_set", "sv_talbanken", "zh_gsdsimp", "zh_gsd"]

    for i in datasets:
        wordsDataTemp, tagsDataTemp = extractData(filePathTrain + i + "-ud-train.iob2")
        wordsData += wordsDataTemp
        tagsData += tagsDataTemp

    wordsDataAnot, tagsDataAnot = Anotation.read_file_and_split("../Data/LotR/LabelingFinals/AllSentences.txt")
    wordsData += wordsDataAnot[0:int(len(wordsDataAnot)*train)]
    tagsData += tagsDataAnot[0:int(len(wordsDataAnot)*train)]
    testWordsData = wordsDataAnot[int(len(wordsDataAnot)*train):]
    testTagsData = tagsDataAnot[int(len(wordsDataAnot)*train):]
    trainData, trainLabels, vocab, labels = convertDataShape(list(zip(wordsData, tagsData)))
    testData, testLabels, _, _ = convertDataShape(list(zip(testWordsData, testTagsData)), training = False, vocabulary = vocab, labels = labels)
    return trainData, trainLabels, vocab, labels, testData, testLabels, testWordsData

In [ ]:
trainDataAll, trainLabelsAll, vocab, labels, testData, testLabels, testWords = loadingAllData()

In [ ]:
testWords

In [ ]:
trainDataEWT = trainDataAll[:-1281]  # 1281 is the floor of 1602 * 0.8, could be substituted by the expression but unsure if we will change it
trainDataLotR = trainDataAll[-1281:]
trainLabelsEWT = trainLabelsAll[:-1281]
trainLabelsLotR = trainLabelsAll[-1281:]

In [ ]:
DIM_EMBEDDING = 300
LSTM_HIDDEN = 100
LEARNING_RATE = 0.01
EPOCHS = 20
BATCH_SIZE = 128
DROPOUT = 0.5


PADDING_TOKEN = '<PAD>'
START_TOKEN = '<START>'
STOP_TOKEN = '<END>'
# The make constraint from the module was yielding some weird results so I decided to hardcode this for our use case, assuming the following dict of tags
#{'<PAD>': 0, '<START>': 1, '<END>': 2, '-': 3, 'B-LOC': 4, 'B-ORG': 5, 'B-OTH': 6, 'B-PER': 7, 'I-LOC': 8, 'I-ORG': 9, 'I-OTH': 10, 'I-PER': 11, 'O': 12}
CONSTRAINTS = [(0, 0), (1, 4), (1, 5), (1, 6), (1, 7), (1, 10), (1, 12), (2, 0), (4, 2), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 12), 
              (5, 2), (5, 4), (5, 5), (5, 6), (5, 7), (5, 9), (5, 12), (6, 2), (6, 4), (6, 5), (6, 6), (6, 7), (6, 10), (6, 12),
              (7, 2), (7, 4), (7, 5), (7, 6), (7, 7), (7, 11), (7, 12), (8, 2), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 12),
              (9, 2), (9, 4), (9, 5), (9, 6), (9, 7), (9, 9), (9, 12), (10, 2), (10, 4), (10, 5), (10, 6), (10, 7), (10, 10), (10, 12),
              (11, 2), (11, 4), (11, 5), (11, 6), (11, 7), (11, 11), (11, 12), (12, 2), (12, 4), (12, 5), (12, 6), (12, 7), (12, 12)]

random.seed(666)
np.random.seed(666)
torch.manual_seed(666)

numBatches = trainDataEWT.shape[0] // BATCH_SIZE

trainDataBatches = trainDataEWT[:BATCH_SIZE*numBatches].view(numBatches, BATCH_SIZE, trainDataEWT.shape[1])
trainLabelsBatches = trainLabelsEWT[:BATCH_SIZE*numBatches].view(numBatches, BATCH_SIZE,  trainLabelsEWT.shape[1])



model = baselineModel(len(vocab), labels, DIM_EMBEDDING, LSTM_HIDDEN, CONSTRAINTS, DROPOUT)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()

    model.zero_grad()

    for batch in zip(trainDataBatches, trainLabelsBatches): 
        optimizer.zero_grad()
        
        loss = model.forwardTrain(batch[0], batch[1])
        loss.backward()
        optimizer.step()
        
     
    print(f"Epoch {epoch}, Loss: {loss.item()}")


In [ ]:
# Constants
NUMBER_OF_MODELS_WITH_FINETUNNING = 10  # Number of models to fine-tune
FINETUNNING_LEARNING_RATE = 0.001
FINETUNNING_EPOCHS = 5
FINETUNNING_BATCH_SIZE = 32  # Batch size

# Assuming trainDataLotR and trainLabelsLotR are defined
finetunningNumBatches = trainDataLotR.shape[0] // FINETUNNING_BATCH_SIZE

# Reshape data into batches
trainDataBatchesLotR = trainDataLotR[:FINETUNNING_BATCH_SIZE * finetunningNumBatches].view(finetunningNumBatches, FINETUNNING_BATCH_SIZE, trainDataLotR.shape[1])
trainLabelsBatchesLotR = trainLabelsLotR[:FINETUNNING_BATCH_SIZE * finetunningNumBatches].view(finetunningNumBatches, FINETUNNING_BATCH_SIZE, trainLabelsLotR.shape[1])

modelBase = model
finetunnedModels = [copy.deepcopy(model) for i in range(NUMBER_OF_MODELS_WITH_FINETUNNING)]

for i, modelToTrain in enumerate(finetunnedModels):
    # Initialize the optimizer with the parameters of the current fine-tuned model
    finetunningOptimizer = torch.optim.Adam(modelToTrain.parameters(), lr=FINETUNNING_LEARNING_RATE)
    
    for epoch in range(FINETUNNING_EPOCHS):
        modelToTrain.train()
        
        j = 0
        cumulative_loss = 0  # To track the cumulative loss for this epoch

        # Train on the appropriate subset of batches for this model
        for batch in zip(trainDataBatchesLotR[: (i + 1) * 4], trainLabelsBatchesLotR[: (i + 1) * 4]):
            finetunningOptimizer.zero_grad()
            loss = modelToTrain.forwardTrain(batch[0], batch[1])
            loss.backward()
            finetunningOptimizer.step()
            
            cumulative_loss += loss.item()
            j += 1

        # Print the average loss for this epoch
        average_loss = cumulative_loss / j
        print(f"Model: {i}, Epoch: {epoch}, Average Loss: {average_loss}")


In [ ]:
#dani Testing  - DELETE
# Constants
FINETUNNING_BATCH_SIZE = 32
model_index = 7  # Model 7 has index 6

# Assuming trainDataLotR and trainLabelsLotR are defined
finetunningNumBatches = trainDataLotR.shape[0] // FINETUNNING_BATCH_SIZE

# Reshape data into batches
trainDataBatchesLotR = trainDataLotR[:FINETUNNING_BATCH_SIZE * finetunningNumBatches].view(finetunningNumBatches, FINETUNNING_BATCH_SIZE, trainDataLotR.shape[1])
trainLabelsBatchesLotR = trainLabelsLotR[:FINETUNNING_BATCH_SIZE * finetunningNumBatches].view(finetunningNumBatches, FINETUNNING_BATCH_SIZE, trainLabelsLotR.shape[1])

# Access data for model 7
model_index = 7
num_batches_for_model = (model_index + 1) * 4  # model 7 trains on 32 batches

data_for_model_7 = trainDataBatchesLotR[:num_batches_for_model]
labels_for_model_7 = trainLabelsBatchesLotR[:num_batches_for_model]

# Print data and labels for model 7
print(f"Data for Model 7 (index {model_index}):")
for batch in data_for_model_7:
    print(batch)
print(f"Labels for Model 7 (index {model_index}):")
for batch in labels_for_model_7:
    print(batch)


In [ ]:
def revertDataShape(Xmatrix, Ymatrix, vocabulary, labels, PADDING_TOKEN='<PAD>', START_TOKEN='<START>', STOP_TOKEN='<END>'):
    """
    Converts matrices of indices back into lists of words and labels.
    
    Input:
    Xmatrix       - 2D torch.tensor                  - 2D torch tensor containing the index of the word in the sentence in the vocabulary
    Ymatrix       - 2D torch.tensor                  - 2D torch tensor containing the index of the label in the sentence in the labels
    vocabulary    - string : int dictionary          - Dictionary of words, with indices as values, used for training.
    labels        - string : int dictionary          - Dictionary of all the labels, with indices as values, used for classification.
    PADDING_TOKEN - string                           - Token used as padding. Default is provided.
    START_TOKEN   - string                           - Token used as marker for the start of the sentence. Default is provided.
    STOP_TOKEN    - string                           - Token used as marker for the end of the sentence. Default is provided.
    
    Output:
    data          - (string list * string list) list - List of sentences. Each sentence is a tuple of two lists. The first is a list of words, the second a list of labels.
    """

    index_to_word = {idx: word for word, idx in vocabulary.items()}
    index_to_label = {idx: label for label, idx in labels.items()}

    data = []
    for i in range(Xmatrix.size(0)):
        words = []
        tags = []
        for j in range(Xmatrix.size(1)):
            word_idx = Xmatrix[i, j].item()
            label_idx = Ymatrix[i, j].item()

            word = index_to_word.get(word_idx, PADDING_TOKEN)
            label = index_to_label.get(label_idx, PADDING_TOKEN)

            # Skip padding, start, and stop tokens
            if word == PADDING_TOKEN or word == START_TOKEN or word == STOP_TOKEN:
                continue
            if label == PADDING_TOKEN or label == START_TOKEN or label == STOP_TOKEN:
                continue

            words.append(word)
            tags.append(label)

        data.append((words, tags))

    return data

# Example usage
reverted_data = revertDataShape(dataDebug, labelsDebug, vocabDebug, tagsDebug)
print(reverted_data)


In [ ]:
trainDataAll, trainLabelsAll, vocab, labels, testData, testLabels, testWords = loadingAllData()

In [ ]:
trainDataEWT = trainDataAll[:-1281]  # 1281 is the floor of 1602 * 0.8, could be substituted by the expression but unsure if we will change it
trainDataLotR = trainDataAll[-1281:]
trainLabelsEWT = trainLabelsAll[:-1281]
trainLabelsLotR = trainLabelsAll[-1281:]

In [ ]:
# getting the results


for i in range(11): 
    
    with torch.no_grad():
        
        if i == 0:
            predicts = model.forwardPred(testData)
        else:
            predicts = finetunnedModels[i-1].forwardPred(testData)

    
    predictLabels = [[list(labels.keys())[i] for i in j if list(labels.keys())[i] != PADDING_TOKEN and list(labels.keys())[i] != START_TOKEN and list(labels.keys())[i] != STOP_TOKEN] for j in predicts]
       
        
    saveToIob2(testWords, predictLabels, "../Results/" + str(i) + ".iob2")
    

        
        

In [ ]:
confMat = MCM(torch.flatten(torch.tensor(labelsDebug, dtype=torch.long)), torch.flatten(labelsDebug), num_classes = len(tagsDebug))

#Calculating Accuracy, Precision, Recall, and F1 Score
#Taking away the first three collumns and rows, because those correspond to the functional tokens and we don't care
acc = torch.trace(confMat[3:,3:])/torch.sum(confMat[3:,3:]) 
precision = torch.diag(confMat[3:, 3:]) / torch.sum(confMat[3:, 3:], dim=1)
recall = torch.diag(confMat[3:, 3:]) / torch.sum(confMat[3:, 3:], dim=0)
f1_score = 2 * (precision * recall) / (precision + recall)
#Taking the average F1 score across all classes
average_f1_score = torch.mean(f1_score)